In [1]:
import tensorflow as tf
import keras
import os

2026-01-16 15:08:36.057581: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-16 15:08:36.064430: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768576116.072146 1841543 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768576116.074432 1841543 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-01-16 15:08:36.082414: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
gpus = tf.config.list_physical_devices(device_type='GPU')
print(gpus)
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
  except RuntimeError as e:
    print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
2 Physical GPUs, 1 Logical GPUs


I0000 00:00:1768576118.216380 1841543 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14240 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:01:00.0, compute capability: 8.6


In [3]:
(training_images, training_labels) , \
(validation_images, validation_labels) = \
tf.keras.datasets.cifar10.load_data()

In [4]:
def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  output_ims = tf.keras.applications.imagenet_utils.preprocess_input(input_images, mode='tf')
  return output_ims

train_X = preprocess_image_input(training_images)
valid_X = preprocess_image_input(validation_images)

In [5]:
@keras.saving.register_keras_serializable()
class ResizeLayer(tf.keras.layers.Layer):
  def __init__(self, size, method='nearest', **kwargs):
    super(ResizeLayer, self).__init__(**kwargs)
    self.size = size
    self.method = method

  def call(self, inputs):
    return tf.image.resize(inputs, size=self.size, method=self.method)

'''
Feature Extraction is performed by ResNet50 pretrained on imagenet weights. 
Input size is 224 x 224.
'''
def feature_extractor(inputs):
  feature_extractor = tf.keras.applications.InceptionV3(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')(inputs)
  return feature_extractor


'''
Defines final dense layers and subsequent softmax layer for classification.
'''
def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Flatten()(x)
    #x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dense(10, activation="softmax", name="classification")(x)
    return x

'''
Since input image size is (32 x 32), first upsample the image by factor of (7x7) to transform it to (224 x 224)
Connect the feature extraction and "classifier" layers to build the model.
'''
def final_model(inputs):
  resize = ResizeLayer(size=(inputs.shape[1] * 7, inputs.shape[2] * 7))(inputs)
  inception_feature_extractor = feature_extractor(resize)
  classification_output = classifier(inception_feature_extractor)
  return classification_output

'''
Define the model and compile it. 
Use Stochastic Gradient Descent as the optimizer.
Use Sparse Categorical CrossEntropy as the loss function.
'''
def define_compile_model():
  inputs = tf.keras.layers.Input(shape=(32,32,3))
  classification_output = final_model(inputs) 
  model = tf.keras.Model(inputs=inputs, outputs = classification_output)
 
  model.compile(optimizer='SGD', 
                loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model


model = define_compile_model()

model.summary()

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 15s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resize_layer (ResizeLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_v3 (Functional)       │ (None, 5, 5, 2048)     │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ classification (Dense)          │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,857,002 (87.19 MB)

 Trainable params: 22,822,570 (87.06 MB)

 Non-trainable params: 34,432 (134.50 KB)

In [6]:
EPOCHS = 3

history = model.fit(train_X, 
                    training_labels, 
                    epochs=EPOCHS, 
                    validation_data = (valid_X, validation_labels), 
                    batch_size=64)

Epoch 1/3


I0000 00:00:1768576177.786590 1841701 service.cc:148] XLA service 0x7fbd5c002060 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1768576177.786765 1841701 service.cc:156]   StreamExecutor device (0): NVIDIA RTX A4000, Compute Capability 8.6
2026-01-16 15:09:38.106213: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1768576178.999261 1841701 cuda_dnn.cc:529] Loaded cuDNN version 90300
2026-01-16 15:09:40.132753: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_14775_0', 204 bytes spill stores, 204 bytes spill loads

2026-01-16 15:09:40.333152: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_14970', 128 bytes

782/782 ━━━━━━━━━━━━━━━━━━━━ 163s 172ms/step - accuracy: 0.7604 - loss: 0.7794 - val_accuracy: 0.9334 - val_loss: 0.2045
Epoch 2/3
782/782 ━━━━━━━━━━━━━━━━━━━━ 118s 151ms/step - accuracy: 0.9578 - loss: 0.1339 - val_accuracy: 0.9498 - val_loss: 0.1530
Epoch 3/3
782/782 ━━━━━━━━━━━━━━━━━━━━ 119s 152ms/step - accuracy: 0.9824 - loss: 0.0582 - val_accuracy: 0.9481 - val_loss: 0.1719


In [7]:
MODEL_PATH = "../models_data/inceptionv3/"
os.makedirs(MODEL_PATH, exist_ok=True)
model.save(MODEL_PATH+'inceptionv3_fp32_cifar10.keras')

In [9]:
model.evaluate(valid_X, validation_labels)
model.evaluate(valid_X[:10000], validation_labels[:10000])

313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.9453 - loss: 0.1852
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.9453 - loss: 0.1852


[0.17191655933856964, 0.9480999708175659]

In [10]:
def representative_dataset():
  for images in tf.data.Dataset.from_tensor_slices(train_X).batch(1).take(1000):
    yield [images]

loaded_model = keras.saving.load_model("../models_data/inceptionv3/inceptionv3_fp32_cifar10.keras")
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT] 
converter.representative_dataset = representative_dataset

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_quant_model = converter.convert()
with open('../models_data/inceptionv3/inceptionv3_uint8_cifar10.tflite', 'wb') as f:
  f.write(tflite_quant_model)

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_quant_model = converter.convert()
with open('../models_data/inceptionv3/inceptionv3_int8_cifar10.tflite', 'wb') as f:
  f.write(tflite_quant_model)

INFO:tensorflow:Assets written to: /tmp/tmphhjrzrua/assets


INFO:tensorflow:Assets written to: /tmp/tmphhjrzrua/assets


Saved artifact at '/tmp/tmphhjrzrua'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  140452550063584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140452550104816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140452550106576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140452550101648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140452550110272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140452550107984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140452550112560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140452550111680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140452550134240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140452550136880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140452550133712:

/home/g.perlo/jacki_venv/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1768576667.962645 1841543 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1768576667.962656 1841543 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2026-01-16 15:17:47.962891: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmphhjrzrua
2026-01-16 15:17:47.969533: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2026-01-16 15:17:47.969544: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmphhjrzrua
I0000 00:00:1768576668.041551 1841543 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2026-01-16 15:17:48.053834: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2026-01-16 15:17:4

INFO:tensorflow:Assets written to: /tmp/tmpmj9jeg1d/assets


INFO:tensorflow:Assets written to: /tmp/tmpmj9jeg1d/assets


Saved artifact at '/tmp/tmpmj9jeg1d'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  140452550063584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140452550104816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140452550106576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140452550101648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140452550110272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140452550107984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140452550112560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140452550111680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140452550134240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140452550136880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140452550133712:

/home/g.perlo/jacki_venv/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1768576753.913528 1841543 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1768576753.913540 1841543 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2026-01-16 15:19:13.913675: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpmj9jeg1d
2026-01-16 15:19:13.923356: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2026-01-16 15:19:13.923368: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpmj9jeg1d
2026-01-16 15:19:14.009045: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2026-01-16 15:19:14.540896: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tm